In [5]:
import json
import pandas as pd

# JSON 파일을 읽어들이기
file_path = '../json/1325171.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터셋에서 필요한 정보 추출
image_info = data.get("이미지 정보", {})
dataset_info = data.get("데이터셋 정보", {})
labeling_info = data.get("라벨링", {})

# 이미지 정보 데이터프레임
image_df = pd.DataFrame([image_info])

# 데이터셋 정보 추출
file_creation_date = dataset_info.get("파일 생성일자")
detailed_description = dataset_info.get("데이터셋 상세설명", {})
rect_coords = detailed_description.get("렉트좌표", {})
polygon_coords = detailed_description.get("폴리곤좌표", {})

# 렉트 좌표와 폴리곤 좌표 데이터프레임
rect_coords_df = pd.DataFrame(rect_coords)
polygon_coords_df = pd.DataFrame(polygon_coords)

# 라벨링 정보 데이터프레임
outer_info = labeling_info.get("아우터", [])
top_info = labeling_info.get("상의", [])
bottom_info = labeling_info.get("하의", [])

# 각 라벨링 정보를 데이터프레임으로 변환
outer_df = pd.DataFrame(outer_info)
top_df = pd.DataFrame(top_info)
bottom_df = pd.DataFrame(bottom_info)

# 결과 출력
print("이미지 정보:")
print(image_df.to_string(index=False))
print("\n파일 생성일자:", file_creation_date)
print("\n렉트 좌표:")
print(rect_coords_df.to_string(index=False))
print("\n폴리곤 좌표:")
print(polygon_coords_df.to_string(index=False))
print("\n아우터 정보:")
print(outer_df.to_string(index=False) if not outer_df.empty else "아우터 정보가 없습니다.")
print("\n상의 정보:")
print(top_df.to_string(index=False) if not top_df.empty else "상의 정보가 없습니다.")
print("\n하의 정보:")
print(bottom_df.to_string(index=False) if not bottom_df.empty else "하의 정보가 없습니다.")


이미지 정보:
 이미지 식별자  이미지 높이                   이미지 파일명  이미지 너비
 1325171     800 fjfcjlwsefkj_3 (1914).JPG     800

파일 생성일자: 2020-11-26 18:51:06

렉트 좌표:
아우터 하의 원피스                                                 상의
 {} {}  {} {'X좌표': 240.203, 'Y좌표': 6.5, '가로': 459, '세로': 527}

폴리곤 좌표:
아우터 하의 원피스                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [15]:
import json
import os
import shutil
from tqdm import tqdm

def find_matching_data(rect_coords):
    """조건에 맞는 데이터를 찾는 함수"""
    has_dress_data = bool(rect_coords.get("원피스") and rect_coords.get("원피스") != [{}])
    has_top_and_bottom_data = bool(rect_coords.get("상의") and rect_coords.get("상의") != [{}] and rect_coords.get("하의") and rect_coords.get("하의") != [{}])
    return has_dress_data or has_top_and_bottom_data

def process_json_files(input_folder, output_folder):
    """폴더 내의 JSON 파일을 순차적으로 처리하고 조건에 맞는 데이터만 저장"""
    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 폴더 내의 모든 JSON 파일 처리
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith(".json"):
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            # 데이터셋 정보 추출
            dataset_info = data.get("데이터셋 정보", {})
            detailed_description = dataset_info.get("데이터셋 상세설명", {})
            rect_coords = detailed_description.get("렉트좌표", {})

            # 조건에 맞는 데이터 찾기
            if find_matching_data(rect_coords):
                # 조건에 맞는 데이터를 새로운 폴더에 저장
                output_file_path = os.path.join(output_folder, filename)
                with open(output_file_path, 'w', encoding='utf-8') as output_file:
                    json.dump(data, output_file, ensure_ascii=False, indent=4)

# 입력 폴더와 출력 폴더 설정
input_folder = '../json/train/매니시'
output_folder = '../matching_json/train/매니시'

# JSON 파일 처리
process_json_files(input_folder, output_folder)


100%|██████████| 28452/28452 [11:06<00:00, 42.66it/s]  
